# Bencharmk communication

In [1]:
from orbita_sdk.serial_io import OrbitaSerialIO

s = OrbitaSerialIO(port='/dev/tty.usbserial-0001', baudrate=500000, timeout=0.1)

In [2]:
s.ping(40)

[]

In [3]:
import time
import numpy as np

l = []

for _ in range(1000):
    t0 = time.time()
    s.ping(40)
    t1 = time.time()
    l.append(t1 - t0)
    
np.mean(l) * 1000, np.std(l) * 1000

(0.5682945251464844, 0.06472786305531437)

In [4]:
from orbita_sdk.register import OrbitaRegister

s.read(40, OrbitaRegister.Zero)

([], [3890, 3566, 1185])

In [5]:
import time
import numpy as np

l = []

for _ in range(1000):
    t0 = time.time()
    s.read(40, OrbitaRegister.PresentPosition)
    t1 = time.time()
    l.append(t1 - t0)
    
np.mean(l) * 1000, np.std(l) * 1000

(0.9266035556793213, 0.07623934660418333)

In [6]:
for reg in OrbitaRegister:
    err, val = s.read(40, reg)
    if err:
        print('ERROR', err)
    print(f'{reg.name}: {val}')

TemperatureShutdown: 50.0
PresentPosition: [-5, -13, 10]
GoalPosition: [0, 0, 0]
TorqueEnable: [True, True, True]
PID: [0.15000000596046448, 0.02500000037252903, 0.016499999910593033]
Temperature: [25.5, 25.8125, 25.3125]
Zero: [3890, 3566, 1185]
Id: 40


In [8]:
print(s.read(40, OrbitaRegister.GoalPosition))
s.write(40, OrbitaRegister.GoalPosition, [10, 0, 30])
print(s.read(40, OrbitaRegister.GoalPosition))

([], [0, 0, 0])
([], [10, 0, 30])


In [9]:
print(s.read(40, OrbitaRegister.PID))
s.write(40, OrbitaRegister.PID, [[1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [1.5, 0.0001, 0.0]])
print(s.read(40, OrbitaRegister.PID))

([], [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, 0.0]])
([], [[1.0, 0.0, 0.0], [2.0, 0.0, 0.0], [1.5, 9.999999747378752e-05, 0.0]])


In [10]:
print(s.read(40, OrbitaRegister.TorqueEnable))
s.write(40, OrbitaRegister.TorqueEnable, [False, True, False])
print(s.read(40, OrbitaRegister.TorqueEnable))

([], [False, False, False])
([], [False, True, False])


In [11]:
s.write(40, OrbitaRegister.TorqueEnable, [True, True, True])

[]

In [12]:
s.read(40, OrbitaRegister.Temperature)

([], [0.0, 0.0, 0.0])

In [13]:
l = []

t0 = time.time()

for _ in range(100):
    s.write(40, OrbitaRegister.GoalPosition, [10, 0, 30])
    s.read(40, OrbitaRegister.PresentPosition)

    t = time.time()
    dt = t - t0

    sleep_time = (0.01 - dt) - 0.001

    if sleep_time > 0:
        time.sleep(sleep_time)
    
    t0 = time.time()
    l.append(time.time())

In [7]:
l = []

for _ in range(10000):
    t0 = time.time()
    s.write(40, OrbitaRegister.GoalPosition, [10, 0, 30])
    s.read(40, OrbitaRegister.PresentPosition)
    t1 = time.time()
    
    l.append(t1 - t0)

dt = np.array(l) * 1000
dt.mean(), dt.std()

(1.8284246921539307, 0.09776431077995759)